In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()

In [3]:
tf.executing_eagerly()
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

hello, [[4.]]


In [4]:
a = tf.constant([[1, 2], [3, 4]])
print(a)

b = tf.add(a, 1)
print(b)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [5]:
import numpy as np

c = np.multiply(a, b)
print(c)
print(a.numpy())

[[ 2  6]
 [12 20]]
[[1 2]
 [3 4]]


In [6]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    for num in range(max_num):
        num = tf.constant(num)
        if int(num % 3) == 0 and int(num % 5) == 0:
            print('FizzBuzz')
        elif int(num % 3) == 0:
            print('Fizz')
        elif int(num % 5) == 0:
            print('Buzz')
        else:
            print(num)
        counter += 1
    return counter

In [7]:
class MySimpleLayer(tf.keras.layers.Layer):
  def __init__(self, output_units):
    self.output_units = output_units

  def build(self, input):
    # The build method gets called the first time your layer is used.
    # Creating variables on build() allows you to make their shape depend
    # on the input shape and hence remove the need for the user to specify
    # full shapes. It is possible to create variables during __init__() if
    # you already know their full shapes.
    self.kernel = self.add_variable(
      "kernel", [input.shape[-1], self.output_units])

  def call(self, input):
    # Override call() instead of __call__ so we can perform some bookkeeping.
    return tf.matmul(input, self.kernel)

In [8]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(784,)),  # must declare input shape
  tf.keras.layers.Dense(10)
])

In [9]:
class MNISTModel(tf.keras.Model):
  def __init__(self):
    super(MNISTModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(units=10)
    self.dense2 = tf.keras.layers.Dense(units=10)

  def call(self, input):
    """Run the model."""
    result = self.dense1(input)
    result = self.dense2(result)
    result = self.dense2(result)  # reuse variables from dense2 layer
    return result

model = MNISTModel()

In [10]:
tfe = tf.contrib.eager

In [11]:
w = tfe.Variable([[1.0]])
with tf.GradientTape() as tape:
  loss = w * w

grad = tape.gradient(loss, [w])
print(grad)  # => [tf.Tensor([[ 2.]], shape=(1, 1), dtype=float32)]

[<tf.Tensor: id=76, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>]


In [12]:
from tqdm import tqdm
for i in tqdm(range(10000)):
  pass

100%|██████████| 10000/10000 [00:00<00:00, 1968509.88it/s]


In [13]:
# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise


def prediction(input, weight, bias):
    return input * weight + bias


# A loss function using mean-squared error
def loss(weights, biases):
    error = prediction(training_inputs, weights, biases) - training_outputs
    return tf.reduce_mean(tf.square(error))


# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value = loss(weights, biases)
    return tape.gradient(loss_value, [weights, biases])


train_steps = 200
learning_rate = 0.01
# Start with arbitrary values for W and B on the same batch of data
W = tfe.Variable(5.)
B = tfe.Variable(10.)

print("Initial loss: {:.3f}".format(loss(W, B)))

for i in range(train_steps):
    dW, dB = grad(W, B)
    W.assign_sub(dW * learning_rate)
    B.assign_sub(dB * learning_rate)
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final loss: {:.3f}".format(loss(W, B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))

Initial loss: 69.164
Loss at step 000: 66.462
Loss at step 020: 30.154
Loss at step 040: 13.981
Loss at step 060: 6.778
Loss at step 080: 3.569
Loss at step 100: 2.140
Loss at step 120: 1.503
Loss at step 140: 1.219
Loss at step 160: 1.093
Loss at step 180: 1.037
Final loss: 1.013
W = 3.0266048908233643, B = 2.135002374649048


In [14]:
dataset = tf.data.Dataset.from_tensor_slices((data.train.images,
                                              data.train.labels))
...
for (batch, (images, labels)) in enumerate(dataset):
    ...
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss_value = loss(logits, labels)
    ...
    grads = tape.gradient(loss_value, model.variables)
    optimizer.apply_gradients(
        zip(grads, model.variables),
        global_step=tf.train.get_or_create_global_step())

NameError: name 'data' is not defined

In [15]:
batch = tf.zeros([1, 1, 784])
print(batch.shape)  # => (1, 1, 784)

result = model(batch)

(1, 1, 784)


In [16]:
import dataset  # download dataset.py file
dataset_train = dataset.train('./datasets').shuffle(60000).repeat(4).batch(32)

In [17]:
def loss(model, x, y):
  prediction = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=prediction)

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

x, y = iter(dataset_train).next()
print("Initial loss: {:.3f}".format(loss(model, x, y)))

# Training loop
for (i, (x, y)) in enumerate(dataset_train):
  # Calculate derivatives of the input function with respect to its parameters.
  grads = grad(model, x, y)
  # Apply the gradient to the model
  optimizer.apply_gradients(zip(grads, model.variables),
                            global_step=tf.train.get_or_create_global_step())
  if i % 200 == 0:
    print("Loss at step {:04d}: {:.3f}".format(i, loss(model, x, y)))

print("Final loss: {:.3f}".format(loss(model, x, y)))

Initial loss: 2.323
Loss at step 0000: 2.334
Loss at step 0200: 2.125
Loss at step 0400: 2.046
Loss at step 0600: 1.969
Loss at step 0800: 1.930
Loss at step 1000: 1.826
Loss at step 1200: 1.879
Loss at step 1400: 1.770
Loss at step 1600: 1.593
Loss at step 1800: 1.629
Loss at step 2000: 1.306
Loss at step 2200: 1.324
Loss at step 2400: 1.300
Loss at step 2600: 1.262
Loss at step 2800: 1.315
Loss at step 3000: 1.058
Loss at step 3200: 1.025
Loss at step 3400: 1.096
Loss at step 3600: 0.831
Loss at step 3800: 0.948
Loss at step 4000: 0.826
Loss at step 4200: 1.062
Loss at step 4400: 0.963
Loss at step 4600: 0.960
Loss at step 4800: 0.768
Loss at step 5000: 0.712
Loss at step 5200: 0.629
Loss at step 5400: 0.645
Loss at step 5600: 0.651
Loss at step 5800: 0.749
Loss at step 6000: 0.658
Loss at step 6200: 0.770
Loss at step 6400: 0.959
Loss at step 6600: 0.530
Loss at step 6800: 0.485
Loss at step 7000: 0.622
Loss at step 7200: 0.474
Loss at step 7400: 0.507
Final loss: 0.649


In [19]:
x = tfe.Variable(10.)

checkpoint = tfe.Checkpoint(x=x)  # save as "x"

x.assign(2.)  # Assign a new value to the variables and save.
save_path = checkpoint.save('./ckpt/')

x.assign(11.)  # Change the variable after saving.

# Restore values from the checkpoint
checkpoint.restore(save_path)

print(x)  # => 2.0

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


In [21]:
model = MyModel()
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
checkpoint_dir = '/path/to/model_dir'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tfe.Checkpoint(
    optimizer=optimizer,
    model=model,
    optimizer_step=tf.train.get_or_create_global_step())

root.save(file_prefix=checkpoint_prefix)
# or
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

NameError: name 'MyModel' is not defined